# McKinney Chapter 8 - Practice - Sec 03

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import yfinance as yf

In [2]:
%precision 4
pd.options.display.float_format = '{:.4f}'.format
%config InlineBackend.figure_format = 'retina'

## Announcements

1. ***The deadline for forming project groups is Tuesday, 2/11.*** That evening, I will create random project groups from the unassigned students.
2. ***The deadline for proposing (and voting on) students' choice topics is Tuesday, 2/25.*** That evening, I will finalize our schedule for the second half of the semester.

## Five-Minute Review

Chapter 8 of McKinney covers 3 important topics.

1. ***Hierarchical Indexing:***
Hierarchical indexes (or multi-indexes) organize data at multiple levels instead of just a flat, two-dimensional structure.
They help us work with high-dimensional data in a low-dimensional form.
For example, we can index rows by multiple levels like `Ticker` and `Date`, or columns by `Variable` and `Ticker`.
2. ***Combining Data:***
We will use three functions and methods to combine datasets on one or more keys.
All three offer `inner`, `outer`, `left`, or `right` combinations.
    1. The `pd.merge()` function (or the `.merge()` method) provides the most flexible way to perform database-style joins on data frames.
    2. The `.join()` method combines data frames with similar indexes.
    3. The `pd.concat()` function combines similarly-shaped series and data frames.
3. ***Reshaping Data:***
We can reshape data to change its structure, such as pivoting from wide to long format or vice versa.
We will most often use the `.stack()` and `.unstack()` methods, which pivot columns to rows and rows to columns, respectively.
Laster in the course we will learn about the `.pivot()` method for aggregating data and the `.melt()` method for more advanced reshaping.

## Practice

### Download data from Yahoo! Finance for BAC, C, GS, JPM, MS, and PNC and assign to data frame `stocks_wide`.

In [3]:
stocks_wide = yf.download(tickers='BAC, C, GS, JPM, MS, PNC', auto_adjust=False, progress=False)

In [4]:
stocks_wide.tail()

Price      Adj Close                                               Close  \
Ticker           BAC       C       GS      JPM       MS      PNC     BAC   
Date                                                                       
2025-01-29   46.7500 80.0755 637.3800 266.5800 137.7931 202.1800 46.7500   
2025-01-30   46.7200 81.2970 645.7000 268.2300 139.0150 202.0700 46.7200   
2025-01-31   46.3000 80.8700 640.4000 267.3000 138.4300 200.9500 46.3000   
2025-02-03   46.2100 79.6100 632.3700 266.8100 137.1600 197.6400 46.2100   
2025-02-04   46.7100 78.4800 634.1800 267.9400 136.7700 198.9800 46.7100   

Price                                 ...     Open                             \
Ticker           C       GS      JPM  ...       GS      JPM       MS      PNC   
Date                                  ...                                       
2025-01-29 80.6300 637.3800 266.5800  ... 637.0000 267.2100 138.4700 201.7200   
2025-01-30 81.8600 645.7000 268.2300  ... 644.4900 268.6300 139.8100 204.2200   
2025-01-31 81.4300 640.4000 267.3000  ... 650.0000 269.2300 139.1400 202.1100   
2025-02-03 79.6100 632.3700 266.8100  ... 626.0000 261.8300 135.8100 197.1900   
2025-02-04 78.4800 634.1800 267.9400  ... 632.0000 269.8400 137.7300 198.0600   

Price         Volume                                                       \
Ticker           BAC             C           GS          JPM           MS   
Date                                                                        
2025-01-29  23681500 10573000.0000 1918900.0000 7684900.0000 5719800.0000   
2025-01-30  31199900 10775500.0000 1812800.0000 8753500.0000 4545700.0000   
2025-01-31  30392900 12821400.0000 1996200.0000 7196300.0000 5503400.0000   
2025-02-03  36481700 20984000.0000 2099200.0000 8376600.0000 5305400.0000   
2025-02-04  23698116  9791535.0000 1429857.0000 4100124.0000 4996375.0000   

Price                    
Ticker              PNC  
Date                     
2025-01-29 2096700.0000  
2025-01-30 1705400.0000  
2025-01-31 2739800.0000  
2025-02-03 1474500.0000  
2025-02-04 1433751.0000  

[5 rows x 36 columns]

### Reshape `stocks_wide` from wide to long with dates and tickers as row indexes and assign to data frame `stocks_long`.

We use the `.stack()` method to go from wider to longer, and the `.unstack()` method to go from long longer to wider.
Note that we set `future_stack=True` to accep the future default arguments for `.stack()` and suppress the `FutureWarning`.
A `FutureWarning` is not an error, just a warning about some expected change that could cause and error in the future.

In [5]:
stocks_long = stocks_wide.stack(future_stack=True)

In [6]:
stocks_long.tail()

Price              Adj Close    Close     High      Low     Open       Volume
Date       Ticker                                                            
2025-02-04 C         78.4800  78.4800  80.2400  78.3600  80.1050 9791535.0000
           GS       634.1800 634.1800 637.6650 629.2500 632.0000 1429857.0000
           JPM      267.9400 267.9400 269.8400 266.8300 269.8400 4100124.0000
           MS       136.7700 136.7700 137.8800 136.2900 137.7300 4996375.0000
           PNC      198.9800 198.9800 200.2788 197.7970 198.0600 1433751.0000

::: {.callout-note}

The `.melt()` methods can reshape data frames from wide to long.
However, our data frame has a column multi-index, which makes `.melt()` difficult to use and `.stack()` a better option.

:::

### Add daily returns to both `stocks_wide` and `stocks_long` under the name `Returns`.

*Hint:* Use `pd.MultiIndex()` to create a multi index for the wide data frame `stocks_wide`.

In [7]:
stocks_wide['Adj Close'].columns

Index(['BAC', 'C', 'GS', 'JPM', 'MS', 'PNC'], dtype='object', name='Ticker')

In [8]:
_ = pd.MultiIndex.from_product([['Returns'], stocks_wide['Adj Close'].columns])

stocks_wide[_] = (
    stocks_wide
    ['Adj Close']
    .iloc[:-1] # do not use mid-day Adj Close for returns calculation
    .pct_change()
)

In [9]:
stocks_wide.tail()

Price      Adj Close                                               Close  \
Ticker           BAC       C       GS      JPM       MS      PNC     BAC   
Date                                                                       
2025-01-29   46.7500 80.0755 637.3800 266.5800 137.7931 202.1800 46.7500   
2025-01-30   46.7200 81.2970 645.7000 268.2300 139.0150 202.0700 46.7200   
2025-01-31   46.3000 80.8700 640.4000 267.3000 138.4300 200.9500 46.3000   
2025-02-03   46.2100 79.6100 632.3700 266.8100 137.1600 197.6400 46.2100   
2025-02-04   46.7100 78.4800 634.1800 267.9400 136.7700 198.9800 46.7100   

Price                                 ...       Volume               \
Ticker           C       GS      JPM  ...           GS          JPM   
Date                                  ...                             
2025-01-29 80.6300 637.3800 266.5800  ... 1918900.0000 7684900.0000   
2025-01-30 81.8600 645.7000 268.2300  ... 1812800.0000 8753500.0000   
2025-01-31 81.4300 640.4000 267.3000  ... 1996200.0000 7196300.0000   
2025-02-03 79.6100 632.3700 266.8100  ... 2099200.0000 8376600.0000   
2025-02-04 78.4800 634.1800 267.9400  ... 1429857.0000 4100124.0000   

Price                                Returns                                  \
Ticker               MS          PNC     BAC       C      GS     JPM      MS   
Date                                                                           
2025-01-29 5719800.0000 2096700.0000 -0.0019  0.0086 -0.0007 -0.0021  0.0012   
2025-01-30 4545700.0000 1705400.0000 -0.0006  0.0153  0.0131  0.0062  0.0089   
2025-01-31 5503400.0000 2739800.0000 -0.0090 -0.0053 -0.0082 -0.0035 -0.0042   
2025-02-03 5305400.0000 1474500.0000 -0.0019 -0.0156 -0.0125 -0.0018 -0.0092   
2025-02-04 4996375.0000 1433751.0000     NaN     NaN     NaN     NaN     NaN   

Price               
Ticker         PNC  
Date                
2025-01-29  0.0023  
2025-01-30 -0.0005  
2025-01-31 -0.0055  
2025-02-03 -0.0165  
2025-02-04     NaN  

[5 rows x 42 columns]

To add returns to `stocks_long` we have two options.
I prefer the first option, but I will present the second option to show an application of the `.join()` method.
I will assign the results of these two options to `stocks_long_1` and `stocks_long_2` so we can keep the original `stocks_long` as-is.

***Option 1:***
Make `stocks_wide` long!

In [10]:
stocks_long_1 = stocks_wide.stack(future_stack=True)

Recall, we omitted returns for the most recent trading day, which could include a partial data return.

In [11]:
stocks_long_1.tail(12)

Price              Adj Close    Close     High      Low     Open  \
Date       Ticker                                                  
2025-02-03 BAC       46.2100  46.2100  46.4800  45.1100  45.4200   
           C         79.6100  79.6100  79.9100  76.8900  78.5000   
           GS       632.3700 632.3700 638.4800 622.4800 626.0000   
           JPM      266.8100 266.8100 268.1700 261.7000 261.8300   
           MS       137.1600 137.1600 138.0000 133.9100 135.8100   
           PNC      197.6400 197.6400 198.7600 194.3000 197.1900   
2025-02-04 BAC       46.7100  46.7100  47.1300  46.2700  46.3500   
           C         78.4800  78.4800  80.2400  78.3600  80.1050   
           GS       634.1800 634.1800 637.6650 629.2500 632.0000   
           JPM      267.9400 267.9400 269.8400 266.8300 269.8400   
           MS       136.7700 136.7700 137.8800 136.2900 137.7300   
           PNC      198.9800 198.9800 200.2788 197.7970 198.0600   

Price                    Volume  Returns  
Date       Ticker                         
2025-02-03 BAC    36481700.0000  -0.0019  
           C      20984000.0000  -0.0156  
           GS      2099200.0000  -0.0125  
           JPM     8376600.0000  -0.0018  
           MS      5305400.0000  -0.0092  
           PNC     1474500.0000  -0.0165  
2025-02-04 BAC    23698116.0000      NaN  
           C       9791535.0000      NaN  
           GS      1429857.0000      NaN  
           JPM     4100124.0000      NaN  
           MS      4996375.0000      NaN  
           PNC     1433751.0000      NaN

***Option 2:***
Calculate returns from `stocks_wide`, make them long, then `.join()` them to `stocks_long`!

In [12]:
_ = stocks_wide['Adj Close'].iloc[:-1].pct_change().stack().to_frame('Returns')

stocks_long_2 = stocks_long.join(_)

Recall, we omitted returns for the most recent trading day, which could include a partial data return.

In [13]:
stocks_long_2.tail(12)

Adj Close    Close     High      Low     Open  \
Date       Ticker                                                  
2025-02-03 BAC       46.2100  46.2100  46.4800  45.1100  45.4200   
           C         79.6100  79.6100  79.9100  76.8900  78.5000   
           GS       632.3700 632.3700 638.4800 622.4800 626.0000   
           JPM      266.8100 266.8100 268.1700 261.7000 261.8300   
           MS       137.1600 137.1600 138.0000 133.9100 135.8100   
           PNC      197.6400 197.6400 198.7600 194.3000 197.1900   
2025-02-04 BAC       46.7100  46.7100  47.1300  46.2700  46.3500   
           C         78.4800  78.4800  80.2400  78.3600  80.1050   
           GS       634.1800 634.1800 637.6650 629.2500 632.0000   
           JPM      267.9400 267.9400 269.8400 266.8300 269.8400   
           MS       136.7700 136.7700 137.8800 136.2900 137.7300   
           PNC      198.9800 198.9800 200.2788 197.7970 198.0600   

                         Volume  Returns  
Date       Ticker                         
2025-02-03 BAC    36481700.0000  -0.0019  
           C      20984000.0000  -0.0156  
           GS      2099200.0000  -0.0125  
           JPM     8376600.0000  -0.0018  
           MS      5305400.0000  -0.0092  
           PNC     1474500.0000  -0.0165  
2025-02-04 BAC    23698116.0000      NaN  
           C       9791535.0000      NaN  
           GS      1429857.0000      NaN  
           JPM     4100124.0000      NaN  
           MS      4996375.0000      NaN  
           PNC     1433751.0000      NaN

We can test the equality of `stocks_long_1` and `stocks_long_2` most easily with the `.equals()` method.

In [14]:
stocks_long_1.equals(stocks_long_2)

True

### Download the daily benchmark return factors from Ken French's data library.

*Hint:*
Use the `DataReader()` function in the pandas-datareader package.
We imported this package above with the `pdr.` prefix.

I often cannot remember the exact name for the daily factors.
We can use the `pdr.famafrench.get_available_datasets()` to list all the data in Kenneth French's data library.

In [15]:
pdr.famafrench.get_available_datasets()[:5]

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily']

In [16]:
ff = pdr.DataReader(
    name='F-F_Research_Data_Factors_daily',
    data_source='famafrench',
    start='1900'
)

C:\Users\r.herron\AppData\Local\Temp\ipykernel_18388\875599436.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.DataReader(


In [17]:
type(ff)

dict

The daily factors only have one data frame (in the `0` key) and the data set description (in the `DESCR` key).

In [18]:
ff.keys()

dict_keys([0, 'DESCR'])

::: {.callout-note}

Data from the Kenneth French data library are *percent* returns instead of *decimal* returns!

:::

In [19]:
ff[0]

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.1000,-0.2500,-0.2700,0.0090
1926-07-02,0.4500,-0.3300,-0.0600,0.0090
1926-07-06,0.1700,0.3000,-0.3900,0.0090
1926-07-07,0.0900,-0.5800,0.0200,0.0090
1926-07-08,0.2100,-0.3800,0.1900,0.0090
...,...,...,...,...
2024-12-24,1.1100,-0.0900,-0.0500,0.0170
2024-12-26,0.0200,1.0400,-0.1900,0.0170
2024-12-27,-1.1700,-0.6600,0.5600,0.0170


In [20]:
print(ff['DESCR'])

F-F Research Data Factors daily
-------------------------------

This file was created by CMPT_ME_BEME_RETS_DAILY using the 202412 CRSP database. The Tbill return is the simple daily rate that, over the number of trading days compounds to 1-month TBill rate. The 1-month TBill rate data until 202405 are from Ibbotson Associates. Starting from 202406, the 1-month TBill rate is from ICE BofA US 1-Month Treasury Bill Index. Copyright 2024 Eugene F. Fama and Kenneth R. French

  0 : (25901 rows x 4 cols)


### Add the daily benchmark return factors to `stocks_wide` and `stocks_long`.

Since both `ff[0]` and `stocks_long_2` have date indexes, we can easily combine them with the `.join()` method.

In [21]:
ff[0].tail()

,Mkt-RF,SMB,HML,RF
Date,,,,
2024-12-24,1.1100,-0.0900,-0.0500,0.0170
2024-12-26,0.0200,1.0400,-0.1900,0.0170
2024-12-27,-1.1700,-0.6600,0.5600,0.0170
2024-12-30,-1.0900,0.1200,0.7400,0.0170
2024-12-31,-0.4600,0.0000,0.7100,0.0170


In [22]:
stocks_long_2.tail(12)

Adj Close    Close     High      Low     Open  \
Date       Ticker                                                  
2025-02-03 BAC       46.2100  46.2100  46.4800  45.1100  45.4200   
           C         79.6100  79.6100  79.9100  76.8900  78.5000   
           GS       632.3700 632.3700 638.4800 622.4800 626.0000   
           JPM      266.8100 266.8100 268.1700 261.7000 261.8300   
           MS       137.1600 137.1600 138.0000 133.9100 135.8100   
           PNC      197.6400 197.6400 198.7600 194.3000 197.1900   
2025-02-04 BAC       46.7100  46.7100  47.1300  46.2700  46.3500   
           C         78.4800  78.4800  80.2400  78.3600  80.1050   
           GS       634.1800 634.1800 637.6650 629.2500 632.0000   
           JPM      267.9400 267.9400 269.8400 266.8300 269.8400   
           MS       136.7700 136.7700 137.8800 136.2900 137.7300   
           PNC      198.9800 198.9800 200.2788 197.7970 198.0600   

                         Volume  Returns  
Date       Ticker                         
2025-02-03 BAC    36481700.0000  -0.0019  
           C      20984000.0000  -0.0156  
           GS      2099200.0000  -0.0125  
           JPM     8376600.0000  -0.0018  
           MS      5305400.0000  -0.0092  
           PNC     1474500.0000  -0.0165  
2025-02-04 BAC    23698116.0000      NaN  
           C       9791535.0000      NaN  
           GS      1429857.0000      NaN  
           JPM     4100124.0000      NaN  
           MS      4996375.0000      NaN  
           PNC     1433751.0000      NaN

We can quickly combine `stocks_long_2` and `ff[0]` because both have indexes with daily dates named `Date`. 
Two notes:

1. The `.join()` method left joins by default, so the combined output has only dates in `stocks_long_2`
2. Kenneth French provides *percent* returns, so we divide them by 100 to convert them to *decimal* returns to match our Yahoo! Finance data

In [23]:
stocks_long_2.join(ff[0].div(100))

Adj Close    Close     High      Low     Open       Volume  \
Date       Ticker                                                               
1973-02-21 BAC        1.5426   4.6250   4.6250   4.6250   4.6250   99200.0000   
           C             NaN      NaN      NaN      NaN      NaN          NaN   
           GS            NaN      NaN      NaN      NaN      NaN          NaN   
           JPM           NaN      NaN      NaN      NaN      NaN          NaN   
           MS            NaN      NaN      NaN      NaN      NaN          NaN   
...                      ...      ...      ...      ...      ...          ...   
2025-02-04 C         78.4800  78.4800  80.2400  78.3600  80.1050 9791535.0000   
           GS       634.1800 634.1800 637.6650 629.2500 632.0000 1429857.0000   
           JPM      267.9400 267.9400 269.8400 266.8300 269.8400 4100124.0000   
           MS       136.7700 136.7700 137.8800 136.2900 137.7300 4996375.0000   
           PNC      198.9800 198.9800 200.2788 197.7970 198.0600 1433751.0000   

                   Returns  Mkt-RF     SMB    HML     RF  
Date       Ticker                                         
1973-02-21 BAC         NaN -0.0074 -0.0039 0.0054 0.0002  
           C           NaN -0.0074 -0.0039 0.0054 0.0002  
           GS          NaN -0.0074 -0.0039 0.0054 0.0002  
           JPM         NaN -0.0074 -0.0039 0.0054 0.0002  
           MS          NaN -0.0074 -0.0039 0.0054 0.0002  
...                    ...     ...     ...    ...    ...  
2025-02-04 C           NaN     NaN     NaN    NaN    NaN  
           GS          NaN     NaN     NaN    NaN    NaN  
           JPM         NaN     NaN     NaN    NaN    NaN  
           MS          NaN     NaN     NaN    NaN    NaN  
           PNC         NaN     NaN     NaN    NaN    NaN  

[78606 rows x 11 columns]

---

We could instead convert the Yahoo! Finance *decimal* returns to *percent* returns.
I do not have a strong preference on all decimal returns or all percent returns, but all returns should have the same form.

In [24]:
(
    stocks_long_2
    .assign(Returns=lambda x: 100 * x['Returns'])
    .join(ff[0])
)

Adj Close    Close     High      Low     Open       Volume  \
Date       Ticker                                                               
1973-02-21 BAC        1.5426   4.6250   4.6250   4.6250   4.6250   99200.0000   
           C             NaN      NaN      NaN      NaN      NaN          NaN   
           GS            NaN      NaN      NaN      NaN      NaN          NaN   
           JPM           NaN      NaN      NaN      NaN      NaN          NaN   
           MS            NaN      NaN      NaN      NaN      NaN          NaN   
...                      ...      ...      ...      ...      ...          ...   
2025-02-04 C         78.4800  78.4800  80.2400  78.3600  80.1050 9791535.0000   
           GS       634.1800 634.1800 637.6650 629.2500 632.0000 1429857.0000   
           JPM      267.9400 267.9400 269.8400 266.8300 269.8400 4100124.0000   
           MS       136.7700 136.7700 137.8800 136.2900 137.7300 4996375.0000   
           PNC      198.9800 198.9800 200.2788 197.7970 198.0600 1433751.0000   

                   Returns  Mkt-RF     SMB    HML     RF  
Date       Ticker                                         
1973-02-21 BAC         NaN -0.7400 -0.3900 0.5400 0.0220  
           C           NaN -0.7400 -0.3900 0.5400 0.0220  
           GS          NaN -0.7400 -0.3900 0.5400 0.0220  
           JPM         NaN -0.7400 -0.3900 0.5400 0.0220  
           MS          NaN -0.7400 -0.3900 0.5400 0.0220  
...                    ...     ...     ...    ...    ...  
2025-02-04 C           NaN     NaN     NaN    NaN    NaN  
           GS          NaN     NaN     NaN    NaN    NaN  
           JPM         NaN     NaN     NaN    NaN    NaN  
           MS          NaN     NaN     NaN    NaN    NaN  
           PNC         NaN     NaN     NaN    NaN    NaN  

[78606 rows x 11 columns]

---

With `stocks_wide`, we have to do a little more work becuase of its column multi-index!
We will use the `pd.MultiIndex.from_product()` trick from above.

In [25]:
_ = pd.MultiIndex.from_product([['Factors'], ff[0].columns])
stocks_wide[_] = ff[0].div(100)

In [26]:
stocks_wide.loc[:'2024'].tail()

Price      Adj Close                                               Close  \
Ticker           BAC       C       GS      JPM       MS      PNC     BAC   
Date                                                                       
2024-12-24   44.3800 70.5117 582.7900 241.0650 126.2201 192.4933 44.3800   
2024-12-26   44.5500 70.8593 581.2300 241.8907 127.1837 193.1777 44.5500   
2024-12-27   44.3400 70.5117 576.1800 239.9308 125.9221 191.6999 44.3400   
2024-12-30   43.9100 69.9059 573.5500 238.0904 124.9188 190.9560 43.9100   
2024-12-31   43.9500 69.9059 572.6200 238.4783 124.8890 191.2734 43.9500   

Price                                 ... Returns                          \
Ticker           C       GS      JPM  ...     BAC       C      GS     JPM   
Date                                  ...                                   
2024-12-24 71.0000 582.7900 242.3100  ...  0.0112  0.0176  0.0210  0.0164   
2024-12-26 71.3500 581.2300 243.1400  ...  0.0038  0.0049 -0.0027  0.0034   
2024-12-27 71.0000 576.1800 241.1700  ... -0.0047 -0.0049 -0.0087 -0.0081   
2024-12-30 70.3900 573.5500 239.3200  ... -0.0097 -0.0086 -0.0046 -0.0077   
2024-12-31 70.3900 572.6200 239.7100  ...  0.0009  0.0000 -0.0016  0.0016   

Price                      Factors                         
Ticker          MS     PNC  Mkt-RF     SMB     HML     RF  
Date                                                       
2024-12-24  0.0210  0.0050  0.0111 -0.0009 -0.0005 0.0002  
2024-12-26  0.0076  0.0036  0.0002  0.0104 -0.0019 0.0002  
2024-12-27 -0.0099 -0.0077 -0.0117 -0.0066  0.0056 0.0002  
2024-12-30 -0.0080 -0.0039 -0.0109  0.0012  0.0074 0.0002  
2024-12-31 -0.0002  0.0017 -0.0046  0.0000  0.0071 0.0002  

[5 rows x 46 columns]

### Write a function `download()` that accepts tickers and returns a wide data frame of returns with the daily benchmark return factors.

We can even add a `shape` argument to return a wide or long data frame!

We can even add a `shape` argument to return a wide or long data frame!

In [27]:
import warnings

def download(tickers, shape='wide'):
    """
    Download stock price data and Fama-French factors, returning in either 'wide' or 'long' format.

    Parameters:
    - tickers (str or list of str): Stock ticker(s) to download.
    - shape (str): Output format, either 'wide' (default) or 'long'.

    Returns:
    - pd.DataFrame: A DataFrame containing stock prices, returns, and Fama-French factors.
    """
    
    # shape must be wide or long
    if shape not in ['wide', 'long']:
        raise ValueError('Invalid shape: must be "wide" or "long".')

    # Download stock data
    stocks = yf.download(tickers=tickers, auto_adjust=False, progress=False)

    # Download Fama-French factors
    # (suppressing FutureWarning for 'date_parser')
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=FutureWarning)
        factors = pdr.DataReader(
            name='F-F_Research_Data_Factors_daily',
            data_source='famafrench',
            start='1900'
        )[0].div(100) # Convert percentages to decimals

    # Multi-index case
    if isinstance(stocks.columns, pd.MultiIndex):
        # Compute daily returns
        _ = pd.MultiIndex.from_product([['Returns'], stocks['Adj Close'].columns])
        stocks[_] = stocks['Adj Close'].pct_change()

        if shape == 'wide':
            # Add factors with multi-index
            _ = pd.MultiIndex.from_product([['Factors'], factors.columns])
            stocks[_] = factors
            return stocks

        # Convert to long format then add factors
        else:
            return stocks.stack(future_stack=True).join(factors)

    # Single index case
    # (redundant with recent versions of yfinance that always return a multi-index)
    stocks['Returns'] = stocks['Adj Close'].pct_change()
    return stocks.join(factors)


In [28]:
download(tickers='AAPL TSLA')

Price      Adj Close             Close              High               Low  \
Ticker          AAPL     TSLA     AAPL     TSLA     AAPL     TSLA     AAPL   
Date                                                                         
1980-12-12    0.0988      NaN   0.1283      NaN   0.1289      NaN   0.1283   
1980-12-15    0.0937      NaN   0.1217      NaN   0.1222      NaN   0.1217   
1980-12-16    0.0868      NaN   0.1127      NaN   0.1133      NaN   0.1127   
1980-12-17    0.0890      NaN   0.1155      NaN   0.1161      NaN   0.1155   
1980-12-18    0.0915      NaN   0.1189      NaN   0.1194      NaN   0.1189   
...              ...      ...      ...      ...      ...      ...      ...   
2025-01-29  239.3600 389.1000 239.3600 389.1000 239.8600 398.5900 234.0100   
2025-01-30  237.5900 400.2800 237.5900 400.2800 240.7900 412.5000 237.2100   
2025-01-31  236.0000 404.6000 236.0000 404.6000 247.1900 419.9900 233.4400   
2025-02-03  228.0100 383.6800 228.0100 383.6800 231.8300 389.1700 225.7000   
2025-02-04  232.8000 392.2100 232.8000 392.2100 233.1300 393.9900 226.6500   

Price                   Open              Volume               Returns  \
Ticker         TSLA     AAPL     TSLA       AAPL          TSLA    AAPL   
Date                                                                     
1980-12-12      NaN   0.1283      NaN  469033600           NaN     NaN   
1980-12-15      NaN   0.1222      NaN  175884800           NaN -0.0522   
1980-12-16      NaN   0.1133      NaN  105728000           NaN -0.0734   
1980-12-17      NaN   0.1155      NaN   86441600           NaN  0.0248   
1980-12-18      NaN   0.1189      NaN   73449600           NaN  0.0290   
...             ...      ...      ...        ...           ...     ...   
2025-01-29 384.4800 234.1200 395.2100   45486100 68033600.0000  0.0046   
2025-01-30 384.4100 238.6700 410.7800   55658300 98092900.0000 -0.0074   
2025-01-31 401.3400 247.1900 401.5300  101075100 83568200.0000 -0.0067   
2025-02-03 374.3600 229.9900 386.6800   72896300 93385500.0000 -0.0339   
2025-02-04 381.4000 227.2000 383.0000   44489128 56316149.0000  0.0210   

Price              Factors                         
Ticker        TSLA  Mkt-RF     SMB     HML     RF  
Date                                               
1980-12-12     NaN  0.0138 -0.0001 -0.0105 0.0006  
1980-12-15     NaN  0.0011  0.0025 -0.0046 0.0006  
1980-12-16     NaN  0.0071 -0.0075 -0.0047 0.0006  
1980-12-17     NaN  0.0152 -0.0086 -0.0034 0.0006  
1980-12-18     NaN  0.0041  0.0022  0.0126 0.0006  
...            ...     ...     ...     ...    ...  
2025-01-29 -0.0226     NaN     NaN     NaN    NaN  
2025-01-30  0.0287     NaN     NaN     NaN    NaN  
2025-01-31  0.0108     NaN     NaN     NaN    NaN  
2025-02-03 -0.0517     NaN     NaN     NaN    NaN  
2025-02-04  0.0222     NaN     NaN     NaN    NaN  

[11127 rows x 18 columns]

::: {.callout-note}

The `yfinance` package is a powerful tool for downloading market data, financial statements, and analyst estimates from Yahoo! Finance.  
However, because `yfinance` relies on Yahoo! Finance’s API, changes to the API can disrupt its functionality.  

Recently, Yahoo! Finance changed API access to earnings forecasts and announcement dates, so we **cannot complete the earnings announcement exercise I had planned**.  

Instead, I will prepare an alternative set of exercises for us to work on in class on Friday.  
Thank you for your flexibility!  

:::

### Download earnings per share for the stocks in `stocks_long` and combine to a long data frame `earnings`.

Use the `.earnings_dates` method described [here](https://pypi.org/project/yfinance/).
Use `pd.concat()` to combine the result of each the `.earnings_date` data frames and assign them to a new data frame `earnings`.
Name the row indexes `Ticker` and `Date` and swap to match the order of the row index in `stocks_long`.

### Combine `earnings` with the returns from `stocks_long`.

Use the `.earnings_dates` method described [here](https://pypi.org/project/yfinance/).
Use `pd.concat()` to combine the result of each the `.earnings_date` data frames and assign them to a new data frame `earnings`.
Name the row indexes `Ticker` and `Date` and swap to match the order of the row index in `stocks_long`.

### Plot the relation between daily returns and earnings surprises

### Repeat the earnings exercise with the S&P 100 stocks

With more data, we can more clearly see the positive relation between earnings surprises and returns!